<a href="https://colab.research.google.com/github/jaab/AI---Deep-Learning-com-PyTorch-e-Python/blob/Classificacao_Binaria_Base_Brest_Cancer/Project_4_brest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


PROJECTO 4 CLASSIFICAÇÃO BINÁRIA BREST CANCER - CLASSIFICAR SOMENTE UM REGISTO E SALVAR O CLASSIFICADOR

Etapa 1: Importação das bibliotecas

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
import torch
import torch.nn as nn
torch.__version__

'2.1.0+cu121'

Etapa 2 : Base de Dados

In [ ]:
np.random.seed(123)
torch.manual_seed(123)

In [ ]:
previsores = pd.read_csv('entradas_breast.csv')
classe = pd.read_csv('saidas_breast.csv')

In [ ]:
previsores[0]

tensor([1.7990e+01, 1.0380e+01, 1.2280e+02, 1.0010e+03, 1.1840e-01, 2.7760e-01,
        3.0010e-01, 1.4710e-01, 2.4190e-01, 7.8710e-02, 1.0950e+03, 9.0530e-01,
        8.5890e+03, 1.5340e+02, 6.3990e-03, 4.9040e-02, 5.3730e-02, 1.5870e-02,
        3.0030e-02, 6.1930e-03, 2.5380e+01, 1.7330e+01, 1.8460e+02, 2.0190e+03,
        1.6220e-01, 6.6560e-01, 7.1190e-01, 2.6540e-01, 4.6010e-01, 1.1890e-01])

In [ ]:

previsores = torch.tensor(np.array(previsores), dtype=torch.float)
classe = torch.tensor(np.array(classe), dtype=torch.float)

In [ ]:
type(previsores)

torch.Tensor

Etapa 3: Transformação dos dados para tensores

In [ ]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(previsores,classe), batch_size=10, shuffle=True)

Etapa 4: Construção do modelo

In [ ]:
class classificador_torch(nn.Module):
  def __init__(self):
    super().__init__()

    # 30 -> 8 -> 8 -> 1
    self.dense0 = nn.Linear(30,8)
    # Mesmos parametros usados pelo kernel initializer do keras (https://keras.io/initializers/, ver sessão RandomNormal)
    torch.nn.init.normal_(self.dense0.weight, mean=0.0, std=0.05)

    self.dense1 = nn.Linear(8,8)
    torch.nn.init.normal_(self.dense1.weight, mean=0.0, std=0.05)

    self.dense2 = nn.Linear(8,1)
    self.activation = nn.ReLU()
    self.dropout = nn.Dropout(0.2)

    self.output = nn.Sigmoid()

  def forward(self,x):
    x = self.dense0(x)
    x = self.activation(x)
    x = self.dropout(x)
    x = self.dense1(x)
    x = self.activation(x)
    x = self.dropout(x)
    x = self.dense2(x)
    x = self.output(x)
    return x

In [ ]:
classificador = classificador_torch()

In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(classificador.parameters(),lr=0.001, weight_decay=0.0001)

Etapa 5: treinamento do modelo

In [ ]:
for epoch in range(100):
  running_loss=0
  for data in train_loader:
    inputs, labels = data
    #print(inputs)
    #print('-----')
    #print(labels)
    optimizer.zero_grad()

    Outputs = classificador(inputs) # or classificador.forward(inputs)
    #print(Outputs)

    loss = criterion(Outputs,labels)  # valor do erro
    #print(loss)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    print('Época %3d: perda %.5f' % (epoch+1, running_loss/len(train_loader)))

A saída de streaming foi truncada nas últimas 5000 linhas.
Época  13: perda 0.14069
Época  13: perda 0.15152
Época  13: perda 0.15875
Época  13: perda 0.16869
Época  13: perda 0.17569
Época  13: perda 0.18687
Época  13: perda 0.19919
Época  13: perda 0.20681
Época  13: perda 0.21571
Época  13: perda 0.22408
Época  13: perda 0.23211
Época  13: perda 0.23900
Época  13: perda 0.24739
Época  13: perda 0.25501
Época  13: perda 0.26610
Época  13: perda 0.27110
Época  13: perda 0.28235
Época  13: perda 0.29050
Época  13: perda 0.29934
Época  13: perda 0.30690
Época  13: perda 0.32208
Época  13: perda 0.33102
Época  13: perda 0.33954
Época  13: perda 0.34442
Época  13: perda 0.35021
Época  13: perda 0.36018
Época  13: perda 0.36923
Época  13: perda 0.37474
Época  13: perda 0.38378
Época  13: perda 0.39496
Época  13: perda 0.40300
Época  13: perda 0.40756
Época  13: perda 0.41668
Época  13: perda 0.42377
Época  13: perda 0.43271
Época  13: perda 0.44007
Época  13: perda 0.44553
Época  13: perda

Etapa 6: Classificar somente o registo

In [ ]:
novo = torch.tensor([[15.80, 9.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                      0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                      0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.15,
                      0.84, 158, 0.363]], dtype=torch.float)

In [49]:
classificador.eval()

classificador_torch(
  (dense0): Linear(in_features=30, out_features=8, bias=True)
  (dense1): Linear(in_features=8, out_features=8, bias=True)
  (dense2): Linear(in_features=8, out_features=1, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (output): Sigmoid()
)

In [50]:
previsao = classificador(novo)

In [51]:
previsao

tensor([[1.]], grad_fn=<SigmoidBackward0>)

In [52]:
previsao = previsao.detach()  # pegar só a parte do tensor

In [53]:
previsao

tensor([[1.]])

In [54]:
previsao = previsao.numpy()

In [73]:
type(previsao)

numpy.ndarray

In [74]:
previsao

array([[False]])

In [79]:
previsao = (previsao > 0.5)

In [80]:
previsao

array([[ True]])

Etapa 7: Salvar o classificador

In [82]:
classificador.state_dict()

OrderedDict([('dense0.weight',
              tensor([[-6.3329e-01, -6.4088e-03, -3.8681e-01, -1.0687e-01,  9.4179e-02,
                       -2.8071e-02,  5.8217e-02, -5.5828e-02,  1.4919e-02,  5.5338e-02,
                        8.5171e-03, -1.0727e-03,  3.1089e-03, -1.7085e-02,  3.1640e-02,
                        2.3500e-01, -3.9340e-01, -1.7852e-01, -6.7033e-02,  5.4838e-01,
                       -4.7472e-01,  2.4388e-02, -3.2421e-01,  1.8188e-01,  1.4345e-02,
                        3.0411e-02,  1.2222e-02,  4.2423e-02,  1.6629e-02,  6.2451e-01],
                      [ 9.6147e-02,  8.9823e-03,  6.2657e-02, -2.7833e-02,  2.5520e-02,
                       -1.4683e-02,  8.4852e-09, -5.5642e-02, -4.3206e-02,  2.9459e-04,
                        3.3087e-02,  1.5457e-03, -3.6564e-02,  7.3219e-02, -9.4868e-11,
                       -5.2539e-07,  3.7381e-03,  2.0600e-09,  6.6002e-09,  2.5737e-10,
                       -7.0341e-02,  3.3297e-02, -3.6295e-02, -3.3981e-02, -7.6689e-02,


In [83]:
torch.save(classificador.load_state_dict, 'classificador.pth')